# Collecting Medicare DRG Payment Data

Using data extracted from: 
- https://jamanetwork.com/journals/jamainternalmedicine/fullarticle/2442504
  - https://jamanetwork.com/data/Journals/INTEMED/934633/IOI150074supp1_prod.pdf

In [3]:
import pandas as pd

In [4]:
raw_data = pd.read_csv("low_value_cpt_codes.csv")
raw_data.columns = [x.strip() for x in raw_data.columns]
raw_data.head(5)

,Qualifications,CPT Codes,Beneficiaries qualifying for potential use of service
0,Cancer screening for patients with chronic kid...,77057 G0202 G0104 G0105 G0106 G0120 G0121 G012...,Patients with CKD receiving dialysis
1,Cervical cancer screening for women over age 65,G0123 G0124 G0141 G0143 G0144 G0145 G0147 G014...,Women over 65
2,Colorectal cancer screening for adults older t...,G0104 G0105 G0106 G0120 G0121 G0122 G0328 4533...,Patients over 75
3,Prostate-specific antigen (PSA) testing for me...,G0103 84152 84153 84154,Men over 75
4,Bone mineral density testing at frequent inter...,76070 76071 76075 76076 76078 76977 77078 7707...,Patients with osteoporosis


In [8]:
def tidy_split(df, column, sep, keep=False):
    """
    Split the values of a column and expand so the new DataFrame has one split
    value per row. Filters rows where the column is missing.

    Params
    ------
    df : pandas.DataFrame
        dataframe with the column to split and expand
    column : str
        the column to split and expand
    sep : str
        the string used to split the column's values
    keep : bool
        whether to retain the presplit value as it's own row

    Returns
    -------
    pandas.DataFrame
        Returns a dataframe with the same columns as `df`.
    """
    indexes = list()
    new_values = list()
    df = df.dropna(subset=[column])
    for i, presplit in enumerate(df[column].astype(str)):
        values = presplit.split(sep)
        if keep and len(values) > 1:
            indexes.append(i)
            new_values.append(presplit)
        for value in values:
            indexes.append(i)
            new_values.append(value)
    new_df = df.iloc[indexes, :].copy()
    new_df[column] = new_values
    return new_df

In [49]:
clean_data = raw_data.copy(deep=True)
clean_data = tidy_split(clean_data, "CPT Codes", " ")
output_data = pd.DataFrame(x.groupby('CPT Codes')['Qualifications'].apply(';'.join))
output_data.to_json("low_value.json", "index")